# Backtesting and report

In [1]:
import os
import pandas as pd
from orats_backtest_v11 import Backtester
from backtest_report import option_trade_report
import logging

In [2]:

max_dte = 8 # 8 days to expiration
min_rr = 0.8 # min risk/reward ratio
max_spread_pct = 0.15 # max spread percentage of underlying price
deltas = [round(i / 100, 2) for i in range(10, 101, 5)] # delta range for options to be traded
print('max_dte:', max_dte)
print('min_rr:', min_rr)
print('max_spread_pct:', max_spread_pct)
print('deltas:', deltas)

max_dte: 8
min_rr: 0.8
max_spread_pct: 0.15
deltas: [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]


In [3]:
# all the signal files are in the same directory, they are named part_001.csv, part_002.csv, etc.
signal_file_ids = [
    '001', '002',
    '003', '004', '005', '006', '007', '008', '009', '010', 
    '011', '012', '013', '014', '015', '016', '017', '018', '019', '020'
    ]

# run the backtest for each signal file
for signal_file_id in signal_file_ids:
    bt = Backtester(
        signals_path=f"signal-files/part_{signal_file_id}.csv",
        orats_path=f"output/part_{signal_file_id}_entry.csv.gz", # orats entry file
        exit_path="output/exit.csv.gz", # orats exit file
        deltas=deltas,
        max_spread_pct=max_spread_pct,
        min_rr=min_rr,
        max_dte=max_dte
    )

    trades = bt.run()
    if not trades.empty:
        os.makedirs("results", exist_ok=True)
        out_file = f"results/part_{signal_file_id}_trades.csv"
        trades.to_csv(out_file, index=False)
        logging.info("Trade log saved to %s", out_file)


2025-05-22 16:23:20,273 INFO: Signals: signal-files/part_001.csv
2025-05-22 16:23:20,273 INFO: Loading CSVs …
2025-05-22 16:24:12,555 INFO: Trades processed: 1177
2025-05-22 16:24:12,573 INFO: Trade log saved to results/part_001_trades.csv
2025-05-22 16:24:12,573 INFO: Signals: signal-files/part_002.csv
2025-05-22 16:24:12,574 INFO: Loading CSVs …
2025-05-22 16:25:18,210 INFO: Trades processed: 1505
2025-05-22 16:25:18,232 INFO: Trade log saved to results/part_002_trades.csv
2025-05-22 16:25:18,232 INFO: Signals: signal-files/part_003.csv
2025-05-22 16:25:18,232 INFO: Loading CSVs …
2025-05-22 16:26:20,711 INFO: Trades processed: 1419
2025-05-22 16:26:20,730 INFO: Trade log saved to results/part_003_trades.csv
2025-05-22 16:26:20,732 INFO: Signals: signal-files/part_004.csv
2025-05-22 16:26:20,732 INFO: Loading CSVs …
2025-05-22 16:27:21,293 INFO: Trades processed: 1372
2025-05-22 16:27:21,312 INFO: Trade log saved to results/part_004_trades.csv
2025-05-22 16:27:21,313 INFO: Signals: s

In [4]:
# joint all trades into one dataframe
trades = pd.DataFrame()
for filename in os.listdir('results'):
    if filename.endswith('_trades.csv'):
        trades = pd.concat([trades, pd.read_csv(os.path.join('results', filename))], ignore_index=True)
        
# sort trades by signal_id
trades = trades.sort_values(by=['signal_id'])
# save the combined trades to a single file
trades.to_csv('results/all_trades.csv', index=False)

print(trades.head())

      signal_id                  signal_ts ticker direction  delta_target  \
0             2  2023-05-03 16:05:00+00:00     GD      call          0.70   
3157          3  2023-05-03 15:35:00+00:00    MPC      call          0.65   
3155          3  2023-05-03 15:35:00+00:00    MPC      call          0.55   
3158          3  2023-05-03 15:35:00+00:00    MPC      call          0.70   
1             3  2023-05-03 15:35:00+00:00    MPC      call          0.25   

                       quote_ts                      expir  spot_entry  \
0     2023-05-03 16:04:43+00:00  2023-05-05 00:00:00+00:00      211.87   
3157  2023-05-03 15:34:53+00:00  2023-05-05 00:00:00+00:00      111.73   
3155  2023-05-03 15:34:53+00:00  2023-05-05 00:00:00+00:00      111.73   
3158  2023-05-03 15:34:53+00:00  2023-05-05 00:00:00+00:00      111.73   
1     2023-05-03 15:34:53+00:00  2023-05-12 00:00:00+00:00      111.73   

      long_strike  short_strike  width  debit  risk_reward  spot_exit  payoff  \
0          

In [5]:
# print(trades.columns)

# Index(['signal_id', 'signal_ts', 'ticker', 'direction', 'delta_target',
#        'quote_ts', 'expir', 'spot_entry', 'long_strike', 'short_strike',
#        'width', 'debit', 'risk_reward', 'spot_exit', 'payoff', 'pnl',
#        'return'],
#       dtype='object')


In [6]:
report_cards = (
    trades
      .groupby("delta_target")
      .apply(lambda grp: option_trade_report(grp))
      .reset_index()
      .set_index("delta_target")
)

# save the report cards to a single file
report_cards.to_csv('results/report.csv', index=False)

print(report_cards)

                                 Start                       End   Period  \
delta_target                                                                
0.10         2023-05-04 00:00:00+00:00 2025-04-25 00:00:00+00:00 722 days   
0.15         2023-05-04 00:00:00+00:00 2025-04-25 00:00:00+00:00 722 days   
0.20         2023-05-03 00:00:00+00:00 2025-04-25 00:00:00+00:00 723 days   
0.25         2023-05-03 00:00:00+00:00 2025-04-25 00:00:00+00:00 723 days   
0.30         2023-05-03 00:00:00+00:00 2025-04-25 00:00:00+00:00 723 days   
0.35         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   
0.40         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   
0.45         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   
0.50         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   
0.55         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   
0.60         2023-05-03 00:00:00+00:00 2025-05-02 00:00:00+00:00 730 days   

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2684\2287577380.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: option_trade_report(grp))
